# SFE-06.3 — Interactive Navigation · Joint Observer Geometry · Plotly

This notebook runs the SFE-06.2 simulation and loads the hull point arrays into interactive Plotly views. Orbit the 3D geometry to see the burst collapse from a volumetric cloud to a planar structure.

**Run cells top to bottom. Each step produces one interactive figure.**


In [ ]:
# Cell 0 — Install dependencies
!pip install plotly scikit-learn -q
print("Dependencies ready.")


In [ ]:
# Cell 1 — Imports
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
print("Imports OK.")


## Cell 2 — Run simulation (SFE-06.2 core)

Produces `bg`, `burst`, `rec` — numpy arrays of joint observer state `(x_a_meas, x_b_meas, corr_current)` per phase.

Also produces `all_records` — full cycle-indexed list for animation.


In [ ]:
# SFE-06.2 simulation — produces hull_pts_bg, hull_pts_burst, hull_pts_rec
# Adapted for Colab: no Agg backend, returns hull point arrays for interactive viewing.

import numpy as np
import warnings; warnings.filterwarnings('ignore')

# ─── PARAMS ───────────────────────────────────────────────────────────────────
kBT=1.0; gamma=1.0; D_diff=kBT/gamma
dt=0.01; tau_meas=10
x_min, x_max = -50.0, 50.0
Nx = 400
x_grid = np.linspace(x_min, x_max, Nx); dx = x_grid[1]-x_grid[0]
sigma_m=0.40; field_volatility=np.sqrt(2*D_diff*tau_meas*dt)

N_background=1000; N_burst=200; N_recovery=400
N_cycles=N_background+N_burst+N_recovery
k_background=0.0; k_burst=1.0; observer_offset=2.0
z_sigma=1.5; W=40
xa0=0.0; xb0=observer_offset

BUFFER_N=512; gate_multiplier=1.0; gate_window=50
coherence_var_th=0.15; alpha_attract=0.15
min_events_before=10; n_candidates=5; rehearsal_on=True
lambda_coup_init=0.30; lambda_min=0.05; lambda_max=0.80
sigma_memory=1.20; x0=0.0; window_scale=1.0
N_steps=N_cycles*tau_meas

# ─── HELPERS ──────────────────────────────────────────────────────────────────
def _gr(m,s):
    r=np.exp(-0.5*((x_grid-m)/s)**2); return r/np.trapezoid(r,x_grid)

def fp_step(rho,k=0.,x0w=0.):
    Ft=-k*(x_grid-x0w); v=Ft/gamma
    df=np.zeros(Nx+1); ff=np.zeros(Nx+1)
    for i in range(1,Nx):
        vf=0.5*(v[i-1]+v[i])
        df[i]=vf*rho[i-1] if vf>=0 else vf*rho[i]
        ff[i]=D_diff*(rho[i]-rho[i-1])/dx
    rn=np.maximum(rho-(dt/dx)*np.diff(df-ff),0.)
    nm=np.trapezoid(rn,x_grid); return rn/nm if nm>1e-12 else rn

def fp_flux(rho): return D_diff*(-np.gradient(rho,x_grid))

class CircularBuffer:
    def __init__(self,N,cols=3):
        self.N=N;self.cols=cols;self.buf=np.zeros((N,cols),dtype=np.float32);self.filled=self.writes=0
    def push(self,row):
        self.writes+=1;self.buf=np.roll(self.buf,shift=1,axis=0)
        self.buf[0]=np.array(row,dtype=np.float32);self.filled=min(self.filled+1,self.N)
    def get_numpy(self): return self.buf[:self.filled]

class AdaptiveGate:
    def __init__(self,mult,window,floor):
        self.mult=mult;self.window=window;self.floor=floor;self._h=[];self.threshold=floor
    def update(self,s):
        self._h.append(s)
        if len(self._h)>self.window: self._h.pop(0)
        if len(self._h)>=4: self.threshold=max(float(np.std(self._h))*self.mult,self.floor)
    def core_gate(self,s): return s<=self.threshold

class KalmanOU:
    def __init__(self,k=0.0): self.k=k;self.x_hat=0.0;self.P=2*kBT/gamma*tau_meas*dt
    def predict_n(self,n):
        if self.k==0.0: self.P+=n*2*kBT/gamma*dt
        else:
            a=np.exp(-self.k*dt*n);sou=kBT/self.k;self.x_hat*=a;self.P=self.P*a**2+sou*(1-a**2)
    def update(self,z):
        K=self.P/(self.P+sigma_m**2);inn=z-self.x_hat;self.x_hat+=K*inn;self.P*=(1-K);return abs(inn)

class PerceivedField:
    def __init__(self,mx=300): self.s=[];self.w=[];self.mx=mx
    def add(self,xp,w=1.0):
        self.s.append(xp);self.w.append(w)
        if len(self.s)>self.mx: self.s.pop(0);self.w.pop(0)
    def get_rho(self):
        if len(self.s)<2: return _gr(0.,1.)
        rho=np.zeros(Nx);wt=sum(self.w)
        for xp,w in zip(self.s,self.w): rho+=(w/wt)*np.exp(-0.5*((x_grid-xp)/sigma_memory)**2)
        nm=np.trapezoid(rho,x_grid); return rho/nm if nm>1e-12 else rho

class CoherenceCentroid:
    def __init__(self): self.centroid=np.zeros(3);self.n=0
    def update(self,pos): self.n+=1;self.centroid=((self.n-1)*self.centroid+pos)/self.n;return self.centroid.copy()
    def attraction_force_x(self,c): return alpha_attract*(self.centroid[0]-c) if self.n>0 else 0.
    @property
    def is_ready(self): return self.n>=min_events_before

class NullPredictorGate:
    def __init__(self,z_sig):
        self.z_sig=z_sig;self._bg=[];self._all=[]
        self.baseline_mean=None;self.baseline_std=None;self.gate_floor=None;self.frozen=False
    def push_background(self,r): self._bg.append(r);self._all.append(r)
    def freeze(self):
        self.baseline_mean=float(np.mean(self._bg));self.baseline_std=float(np.std(self._bg))+1e-8
        self.gate_floor=self.baseline_mean-self.z_sig*self.baseline_std;self.frozen=True
        return self.baseline_mean,self.baseline_std,self.gate_floor
    def push_active(self,r): self._all.append(r)
    def check(self,r,rv,core_rate,pts,cycle):
        if not self.frozen: return False,False,float('nan')
        delta=r-self.gate_floor;crossing=r<self.gate_floor
        if crossing:
            prev=self._all[-2] if len(self._all)>=2 else r
            verified=prev<self.gate_floor
            struct_ok=(rv<coherence_var_th)and(core_rate>=0.20)
            if verified and struct_ok and len(pts)>=5: return True,True,delta
        return False,False,delta

class CrossCorrGate:
    def __init__(self,z_sig,window=W):
        self.z_sig=z_sig;self.W=window;self._bg=[];self._all=[]
        self.baseline_mean=None;self.baseline_std=None;self.gate_ceil=None;self.frozen=False
    def _corr(self,ra,rb):
        if len(ra)<4: return 0.
        c=np.corrcoef(ra,rb); return float(c[0,1]) if np.isfinite(c[0,1]) else 0.
    def push_background(self,ra,rb): c=self._corr(ra,rb);self._bg.append(c);self._all.append(c);return c
    def freeze(self):
        self.baseline_mean=float(np.mean(self._bg));self.baseline_std=float(np.std(self._bg))+1e-8
        self.gate_ceil=self.baseline_mean+self.z_sig*self.baseline_std;self.frozen=True
        return self.baseline_mean,self.baseline_std,self.gate_ceil
    def push_active(self,ra,rb): c=self._corr(ra,rb);self._all.append(c);return c
    def check(self,c):
        if not self.frozen: return False
        return c>self.gate_ceil
    @property
    def all_corrs(self): return np.array(self._all)

# ─── RUN ──────────────────────────────────────────────────────────────────────
print("Running SFE-06.2 simulation... (~30 seconds)")
rng=np.random.default_rng(42)
rng_obs_a=np.random.default_rng(101); rng_obs_b=np.random.default_rng(202)

kf=KalmanOU(k=k_background); pf=PerceivedField()
cb=CircularBuffer(BUFFER_N); wg=AdaptiveGate(gate_multiplier,gate_window,floor=field_volatility)
cen=CoherenceCentroid()
rho=_gr(0.,2.); x=0.; pf.add(x)
lam=lambda_coup_init; xa=xa0; xb=xb0
gate_a=NullPredictorGate(z_sigma); cc_gate=CrossCorrGate(z_sigma,W)
jcb=CircularBuffer(BUFFER_N,cols=3); current_corr=0.0

# Hull point collectors
hull_pts_bg=[]; hull_pts_burst=[]; hull_pts_rec=[]
# Full cycle-indexed records for animation
all_records=[]   # (xa_meas, xb_meas, corr, phase, cycle_idx)

nr_a_log=[]; nr_b_log=[]; corr_log=[]; wc_log=[]; phase_log=[]
null_sw_a=[]; null_sw_b=[]; kalman_sw=[]
vw=[]; total=0; cw=0; cycle=0; baseline_frozen=False

for i in range(N_steps):
    if   cycle<N_background:           phase='bg';    k_now=k_background
    elif cycle<N_background+N_burst:   phase='burst'; k_now=k_burst
    else:                              phase='rec';   k_now=k_background
    kf.k=k_now

    if i%tau_meas==0:
        kf.predict_n(tau_meas)
        xm_agent=x+sigma_m*rng.standard_normal()
        kalman_innov=kf.update(xm_agent)
        xm_a=xa+sigma_m*rng_obs_a.standard_normal()
        xm_b=xb+sigma_m*rng_obs_b.standard_normal()

        jcb.push([xm_a,xm_b,current_corr])
        pt=[float(xm_a),float(xm_b),float(current_corr)]
        if   phase=='bg':    hull_pts_bg.append(pt)
        elif phase=='burst': hull_pts_burst.append(pt)
        else:                hull_pts_rec.append(pt)
        all_records.append({'xa':float(xm_a),'xb':float(xm_b),'corr':float(current_corr),
                            'phase':phase,'cycle':cycle})

        wg.update(kalman_innov); total+=1
        if wg.core_gate(kalman_innov): cb.push([xm_a,xm_b,current_corr]); cw+=1

        null_sw_a.append(abs(xm_a)); null_sw_b.append(abs(xm_b)); kalman_sw.append(kalman_innov)
        if len(null_sw_a)>W: null_sw_a.pop(0);null_sw_b.pop(0);kalman_sw.pop(0)
        cycle+=1

        if cycle%W==0 and cb.filled>=5:
            T=jcb.get_numpy(); vw.append(0.); wl=max(int(N_cycles*0.5),8)
            if len(vw)>wl: vw.pop(0)
            core_rate=cw/max(total,1)
            if len(vw)>=3:
                vm=np.mean(vw)+1e-8; rv=float(np.var(vw))/(vm**2)
            else: rv=0.
            win_null_a=float(np.mean(null_sw_a))/(field_volatility+1e-10)
            win_null_b=float(np.mean(null_sw_b))/(field_volatility+1e-10)
            nr_a_log.append(win_null_a); nr_b_log.append(win_null_b)
            wc_log.append(cycle); phase_log.append(phase)

            if phase=='bg':
                gate_a.push_background(win_null_a)
                current_corr=cc_gate.push_background(null_sw_a[-W:],null_sw_b[-W:])
                corr_log.append(current_corr)
            else:
                if not baseline_frozen:
                    gate_a.freeze(); cc_gate.freeze(); baseline_frozen=True
                gate_a.push_active(win_null_a)
                current_corr=cc_gate.push_active(null_sw_a[-W:],null_sw_b[-W:])
                corr_log.append(current_corr)

    Fou_a=-k_now*(xa-x0)
    xa+=(Fou_a/gamma)*dt+np.sqrt(2*kBT*gamma)*rng_obs_a.standard_normal()*np.sqrt(dt)/gamma
    Fou_b=-k_now*(xb-x0)
    xb+=(Fou_b/gamma)*dt+np.sqrt(2*kBT*gamma)*rng_obs_b.standard_normal()*np.sqrt(dt)/gamma
    rho=fp_step(rho,k_now,x0)
    J=fp_flux(rho); Jat=float(np.interp(x,x_grid,J))
    Ff=lam*Jat/(abs(Jat)+1e-10); Fou=-k_now*(x-x0)
    Fatt=cen.attraction_force_x(abs(Jat)) if cen.is_ready else 0.
    def step_agent(Fe):
        xi=np.sqrt(2*kBT*gamma)*rng.standard_normal()
        return float(np.clip(x+((Ff+Fe)/gamma)*dt+xi*np.sqrt(dt)/gamma,x_min+0.1,x_max-0.1))
    xs=step_agent(Fatt+Fou); bs=abs(xs-kf.x_hat); bx=xs
    if rehearsal_on and n_candidates>1:
        for _ in range(n_candidates-1):
            xc=step_agent(Fatt+Fou);sc=abs(xc-kf.x_hat)
            if sc<bs: bx=xc;bs=sc
    pf.add(bx,w=1./(sigma_m+.1)); x=bx

# Convert to arrays
bg    = np.array(hull_pts_bg)
burst = np.array(hull_pts_burst)
rec   = np.array(hull_pts_rec)

print(f"Simulation complete.")
print(f"  background: {len(bg)} points")
print(f"  burst:      {len(burst)} points")
print(f"  recovery:   {len(rec)} points")


## Cell 3 — Normalize + PCA

Z-score normalize using background statistics, then fit PCA per phase.
Background eigenvalues should be spread [0.41, 0.39, 0.20].
Burst eigenvalues should concentrate on e1 [0.70, 0.27, 0.04].


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import pandas as pd

# Z-score normalize using background statistics
scaler = StandardScaler()
scaler.fit(bg)

bg_norm    = scaler.transform(bg)
burst_norm = scaler.transform(burst)
rec_norm   = scaler.transform(rec)

# PCA per phase
pca_bg_fit = PCA(n_components=3).fit(bg_norm)
pca_bu_fit = PCA(n_components=3).fit(burst_norm)
pca_rec_fit= PCA(n_components=3).fit(rec_norm)

# Transform all phases into background PCA space (common basis)
bg_pca    = pca_bg_fit.transform(bg_norm)
burst_pca = pca_bg_fit.transform(burst_norm)
rec_pca   = pca_bg_fit.transform(rec_norm)

print("=" * 60)
print("SFE-06.3 — Interactive Geometry Navigation")
print("=" * 60)
print(f"\nPoints collected:")
print(f"  background: {len(bg)}")
print(f"  burst:      {len(burst)}")
print(f"  recovery:   {len(rec)}")

print(f"\nPCA eigenvalues (z-score normalized):")
print(f"  background: {pca_bg_fit.explained_variance_ratio_.round(3)}")
print(f"  burst:      {pca_bu_fit.explained_variance_ratio_.round(3)}")
print(f"  recovery:   {pca_rec_fit.explained_variance_ratio_.round(3)}")

print(f"\nShape classification:")
e1_bg = pca_bg_fit.explained_variance_ratio_[0]
e1_bu = pca_bu_fit.explained_variance_ratio_[0]
shape_bg  = 'VOLUMETRIC' if e1_bg < 0.50 else ('PLANAR' if e1_bg < 0.70 else 'LINEAR')
shape_bu  = 'VOLUMETRIC' if e1_bu < 0.50 else ('PLANAR' if e1_bu < 0.70 else 'LINEAR')
print(f"  background: {shape_bg}  (e1={e1_bg:.3f})")
print(f"  burst:      {shape_bu}  (e1={e1_bu:.3f})")

print(f"\nBurst collapse axis (PC1 in original coordinates):")
print(f"  x_a weight:  {pca_bu_fit.components_[0][0]:.3f}")
print(f"  x_b weight:  {pca_bu_fit.components_[0][1]:.3f}")
print(f"  corr weight: {pca_bu_fit.components_[0][2]:.3f}")
print("=" * 60)


## Step 3 — Raw joint space `(x_a, x_b, corr)`

**What to look for:** Orbit until burst (orange) aligns with the `x_a = x_b` diagonal reference line. Background (teal) fills a loose cloud in all directions.


In [ ]:
import plotly.graph_objects as go
import numpy as np

# Subsample for rendering performance
def sub(arr, n=800):
    if len(arr) <= n: return arr
    idx = np.linspace(0, len(arr)-1, n, dtype=int)
    return arr[idx]

bg_s    = sub(bg,    800)
burst_s = sub(burst, 600)
rec_s   = sub(rec,   600)

# Diagonal reference line at corr=0
diag_range = np.linspace(
    min(bg_s[:,0].min(), bg_s[:,1].min()) * 0.8,
    max(bg_s[:,0].max(), bg_s[:,1].max()) * 0.8,
    50
)

fig = go.Figure()

fig.add_trace(go.Scatter3d(
    x=bg_s[:,0], y=bg_s[:,1], z=bg_s[:,2],
    mode='markers',
    marker=dict(size=2, color='#3dd6c8', opacity=0.35),
    name='background'
))
fig.add_trace(go.Scatter3d(
    x=burst_s[:,0], y=burst_s[:,1], z=burst_s[:,2],
    mode='markers',
    marker=dict(size=4, color='#ff8c42', opacity=0.85),
    name='burst'
))
fig.add_trace(go.Scatter3d(
    x=rec_s[:,0], y=rec_s[:,1], z=rec_s[:,2],
    mode='markers',
    marker=dict(size=2, color='#b87aff', opacity=0.35),
    name='recovery'
))
fig.add_trace(go.Scatter3d(
    x=diag_range, y=diag_range, z=np.zeros(50),
    mode='lines',
    line=dict(color='white', width=2, dash='dash'),
    name='x_a = x_b diagonal'
))

fig.update_layout(
    title=dict(
        text='SFE-06.3 · Step 3 — Joint Observer Space (x_a, x_b, corr)<br>'
             '<sup>Orbit to see burst (orange) collapse toward diagonal</sup>',
        font=dict(color='white')
    ),
    scene=dict(
        xaxis_title='x_a measurement',
        yaxis_title='x_b measurement',
        zaxis_title='correlation',
        bgcolor='black',
        xaxis=dict(backgroundcolor='#0a0a0a', gridcolor='#333', color='#aaa'),
        yaxis=dict(backgroundcolor='#0a0a0a', gridcolor='#333', color='#aaa'),
        zaxis=dict(backgroundcolor='#0a0a0a', gridcolor='#333', color='#aaa'),
    ),
    paper_bgcolor='#07080f',
    font=dict(color='white'),
    legend=dict(bgcolor='#0a0a0a', bordercolor='#333'),
    width=900, height=700
)

fig.show()
print("\nStep 3: Raw joint space. Orbit until burst (orange) aligns with x_a=x_b diagonal.")


## Step 4 — PCA space (background basis)

**What to look for:** Background looks like a sphere — variance spread across PC1, PC2, PC3. Burst looks like a pancake — compressed into the PC1 axis. Orbit until you see the background sphere and burst ellipsoid from the same viewpoint.


In [ ]:
import plotly.graph_objects as go
import numpy as np

def sub(arr, n=800):
    if len(arr) <= n: return arr
    idx = np.linspace(0, len(arr)-1, n, dtype=int)
    return arr[idx]

bg_p    = sub(bg_pca,    800)
burst_p = sub(burst_pca, 600)
rec_p   = sub(rec_pca,   600)

fig2 = go.Figure()

fig2.add_trace(go.Scatter3d(
    x=bg_p[:,0], y=bg_p[:,1], z=bg_p[:,2],
    mode='markers',
    marker=dict(size=2, color='#3dd6c8', opacity=0.35),
    name='background'
))
fig2.add_trace(go.Scatter3d(
    x=burst_p[:,0], y=burst_p[:,1], z=burst_p[:,2],
    mode='markers',
    marker=dict(size=4, color='#ff8c42', opacity=0.85),
    name='burst'
))
fig2.add_trace(go.Scatter3d(
    x=rec_p[:,0], y=rec_p[:,1], z=rec_p[:,2],
    mode='markers',
    marker=dict(size=2, color='#b87aff', opacity=0.35),
    name='recovery'
))

# PC axis arrows from origin
axis_len = float(np.abs(bg_pca).max()) * 0.6
for i, (label, color) in enumerate(zip(
    ['PC1 (dominant)', 'PC2', 'PC3'],
    ['#ff5f7e', '#f5c842', '#60a5fa']
)):
    vec = [0, 0, 0]
    vec[i] = axis_len
    fig2.add_trace(go.Scatter3d(
        x=[0, vec[0]], y=[0, vec[1]], z=[0, vec[2]],
        mode='lines+text',
        line=dict(color=color, width=5),
        text=['', label],
        textposition='top center',
        textfont=dict(color=color, size=11),
        name=label
    ))

fig2.update_layout(
    title=dict(
        text='SFE-06.3 · Step 4 — PCA Space (background basis)<br>'
             '<sup>Burst collapses along PC1 — sphere → pancake transition</sup>',
        font=dict(color='white')
    ),
    scene=dict(
        xaxis_title='PC1',
        yaxis_title='PC2',
        zaxis_title='PC3',
        bgcolor='black',
        xaxis=dict(backgroundcolor='#0a0a0a', gridcolor='#333', color='#aaa'),
        yaxis=dict(backgroundcolor='#0a0a0a', gridcolor='#333', color='#aaa'),
        zaxis=dict(backgroundcolor='#0a0a0a', gridcolor='#333', color='#aaa'),
    ),
    paper_bgcolor='#07080f',
    font=dict(color='white'),
    legend=dict(bgcolor='#0a0a0a', bordercolor='#333'),
    width=900, height=700
)

fig2.show()
print("\nStep 4: PCA space. Background should look like a sphere. Burst like a pancake along PC1.")
print(f"  Background e1={pca_bg_fit.explained_variance_ratio_[0]:.3f} — spread across axes")
print(f"  Burst      e1={pca_bu_fit.explained_variance_ratio_[0]:.3f} — concentrated on PC1")


## Step 5 — Time-resolved animation

**What to watch:** Press play. Watch the background cloud expand, then suddenly compress at burst onset (~cycle 1000). Recovery shows the cloud re-expanding as particles re-diffuse from the shared confinement point.


In [ ]:
import plotly.express as px
import pandas as pd

# Build dataframe — subsample for animation performance
# Use every 5th point to keep animation snappy
def sub_df(records, every=5):
    return [r for i, r in enumerate(records) if i % every == 0]

records = all_records  # from simulation
df = pd.DataFrame(sub_df(records, every=5))

# Add cycle-relative label for animation
df['cycle_label'] = df['cycle'].astype(str)

# Color map
color_map = {
    'bg':    '#3dd6c8',
    'burst': '#ff8c42',
    'rec':   '#b87aff'
}

fig3 = px.scatter_3d(
    df, x='xa', y='xb', z='corr',
    color='phase',
    animation_frame='cycle',
    color_discrete_map=color_map,
    title='SFE-06.3 · Step 5 — Time-Resolved Joint Geometry',
    labels={'xa': 'x_a meas', 'xb': 'x_b meas', 'corr': 'correlation'},
    opacity=0.7,
)

fig3.update_traces(marker=dict(size=3))
fig3.update_layout(
    width=900, height=700,
    paper_bgcolor='#07080f',
    font=dict(color='white'),
    scene=dict(
        bgcolor='black',
        xaxis=dict(backgroundcolor='#0a0a0a', gridcolor='#333', color='#aaa'),
        yaxis=dict(backgroundcolor='#0a0a0a', gridcolor='#333', color='#aaa'),
        zaxis=dict(backgroundcolor='#0a0a0a', gridcolor='#333', color='#aaa'),
    )
)

fig3.show()
print("\nStep 5: Animation. Watch the cloud compress at burst onset (cycle ~1000).")
print("Use the play button. Drag to orbit between frames.")


## Step 6 — Eigenvalue spectrum

Static bar chart showing the numeric shape change. Burst e1 crossing the 0.65 threshold is the quantitative confirmation of the volumetric → planar collapse.


In [ ]:
import plotly.graph_objects as go

components = ['e1 (dominant)', 'e2', 'e3']
bg_eig  = pca_bg_fit.explained_variance_ratio_
bu_eig  = pca_bu_fit.explained_variance_ratio_
rec_eig = pca_rec_fit.explained_variance_ratio_

fig4 = go.Figure()

fig4.add_trace(go.Bar(
    name='background', x=components, y=bg_eig,
    marker_color='#3dd6c8', opacity=0.85,
    text=[f'{v:.3f}' for v in bg_eig],
    textposition='outside', textfont=dict(color='#3dd6c8')
))
fig4.add_trace(go.Bar(
    name='burst', x=components, y=bu_eig,
    marker_color='#ff8c42', opacity=0.85,
    text=[f'{v:.3f}' for v in bu_eig],
    textposition='outside', textfont=dict(color='#ff8c42')
))
fig4.add_trace(go.Bar(
    name='recovery', x=components, y=rec_eig,
    marker_color='#b87aff', opacity=0.85,
    text=[f'{v:.3f}' for v in rec_eig],
    textposition='outside', textfont=dict(color='#b87aff')
))

fig4.add_hline(
    y=0.65, line_dash='dot', line_color='#ff5f7e', line_width=1.5,
    annotation_text='dominant threshold (0.65)',
    annotation_font=dict(color='#ff5f7e')
)

fig4.update_layout(
    title=dict(
        text='SFE-06.3 · Step 6 — PCA Eigenvalue Distribution by Phase<br>'
             '<sup>Burst e1 above threshold = collapse to dominant axis confirmed</sup>',
        font=dict(color='white')
    ),
    xaxis=dict(title='Principal Component', color='#aaa', gridcolor='#333'),
    yaxis=dict(title='Explained Variance Ratio', range=[0, 1.05],
               color='#aaa', gridcolor='#333'),
    barmode='group',
    paper_bgcolor='#07080f',
    plot_bgcolor='#0a0a0a',
    font=dict(color='white'),
    legend=dict(bgcolor='#0a0a0a', bordercolor='#333'),
    width=800, height=500
)

fig4.show()

print("\nStep 6: Eigenvalue comparison.")
print(f"  Background e1={bg_eig[0]:.3f} — spread (VOLUMETRIC)")
print(f"  Burst      e1={bu_eig[0]:.3f} — concentrated (PLANAR/LINEAR)")
print(f"  Shape change confirmed: {bg_eig[0]:.3f} → {bu_eig[0]:.3f}")


## Lineage

| Version | Result |
|---------|--------|
| 05.12b  | Showed what a matched filter cannot see |
| 05.13b  | Showed what a single decoupled observer can see |
| SFE-06  | Showed what the relationship between observers can see |
| SFE-06.2 | Showed the shape of that relationship |
| SFE-06.3 | Navigate inside the shape |

---

The information is not in the particles.  
It is in the geometry between them.  
The geometry has shape.  
**SFE-06.3 lets you orbit that shape.**
